# Práctica 2: Aprendizaje y selección de modelos de clasificación

## Minería de Datos

### Curso académico 2021-2022

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

---

**Notas**:

* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo

---

En esta práctica estudiaremos los modelos más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los clasificadores resultantes. Además, veremos métodos de selección de modelos orientados a obtener una configuración óptima de hiperparámetros.

# 1. Preliminares

Antes de comenzar, vamos a fijar una semilla para que los experimentos sean reproducibles:

In [1]:
random_state = 27912

Por último, vamos a suprimir todos las advertencias para evitar salidas demasiado largas a la hora de entrenar los modelos:

In [2]:
import warnings

In [3]:
warnings.filterwarnings("ignore")

# 2. Carga de datos

Como es habitual, utilizaremos el conjunto de datos `iris` para nuestros experimentos:

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("input/iris/Iris.csv", index_col="Id")

In [6]:
target = "Species"

In [7]:
data[target] = data[target].astype("category")

Comprobando que se ha cargado correctamente:

In [8]:
data.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
106,7.6,3.0,6.6,2.1,Iris-virginica
133,6.4,2.8,5.6,2.2,Iris-virginica
132,7.9,3.8,6.4,2.0,Iris-virginica
52,6.4,3.2,4.5,1.5,Iris-versicolor
48,4.6,3.2,1.4,0.2,Iris-setosa


A su vez, lo dividimos en variables predictoras y variable clase:

In [9]:
X = data.drop(target, axis=1)

In [10]:
y = data[target]

Vamos a comprobar que se ha separado correctamente. Comenzamos con las variables predictoras:

In [11]:
X.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Id,,,,
106,7.6,3.0,6.6,2.1
133,6.4,2.8,5.6,2.2
132,7.9,3.8,6.4,2.0
52,6.4,3.2,4.5,1.5
48,4.6,3.2,1.4,0.2


Y continuamos con la variable clase:

In [12]:
y.sample(5, random_state=random_state)

Id
106     Iris-virginica
133     Iris-virginica
132     Iris-virginica
52     Iris-versicolor
48         Iris-setosa
Name: Species, dtype: category
Categories (3, object): ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

Por último, dividimos el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado:

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=random_state, train_size=0.7)

Y nos aseguramos que se ha realizado adecuadamente. Comenzamos con el conjunto de datos de entrenamiento:

In [15]:
X_train.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Id,,,,
49,5.3,3.7,1.5,0.2
33,5.2,4.1,1.5,0.1
66,6.7,3.1,4.4,1.4
63,6.0,2.2,4.0,1.0
111,6.5,3.2,5.1,2.0


In [16]:
y_train.sample(5, random_state=random_state)

Id
49         Iris-setosa
33         Iris-setosa
66     Iris-versicolor
63     Iris-versicolor
111     Iris-virginica
Name: Species, dtype: category
Categories (3, object): ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

Y finalizamos con el conjunto de datos de prueba:

In [17]:
X_test.sample(5, random_state=random_state)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Id,,,,
88,6.3,2.3,4.4,1.3
6,5.4,3.9,1.7,0.4
109,6.7,2.5,5.8,1.8
91,5.5,2.6,4.4,1.2
28,5.2,3.5,1.5,0.2


In [18]:
y_test.sample(5, random_state=random_state)

Id
88     Iris-versicolor
6          Iris-setosa
109     Iris-virginica
91     Iris-versicolor
28         Iris-setosa
Name: Species, dtype: category
Categories (3, object): ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

# 3. Modelos de clasificación supervisada

## 3.1. Vecinos más cercanos

El algoritmo de los vecinos más cercanos es un método basado en instancias que no construye ningún modelo durante el aprendizaje, pues directamente almacena las instancias del conjunto de datos de entrenamiento. Además, se considera perozoso, dado que computa los parámetros necesarios para la clasificación en la fase de inferencia. En particular, la clasificación consiste en asignar a la instancia de entrada la clase mayoritaria de los vecinos más cercanos.

Este clasificador se implementa en la clase [`neighbors.KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). Vamos a inicializar este clasificador usando un número de vecinos más cercanos moderado. Para ello, configuramos el modelo con respecto a los hiperparámetros:

In [19]:
from sklearn.neighbors import KNeighborsClassifier

In [20]:
n_neighbors = 5

In [21]:
k_neighbors_model = KNeighborsClassifier(n_neighbors)

## 3.2. Árboles de decisión

Los algoritmos basados en inducción de árboles de decisión representan los modelos mediante un conjunto de reglas. Estos presentan una serie de ventajas tal y como puede ser la interpretabilidad de los modelos obtenidos, bajo coste del proceso de predicción (logarítmico con respecto al número de muestras del conjunto de datos de prueba), manejo implícito de variables numéricas y categóricas, etc. Por contra, una de las mayores desventajas es que tienden a crear modelos demasiado complejos que no suelen generalizar ante conjuntos de datos no visualizados por el algoritmo de aprendizaje. Entre otros problemas destacan la inestabilidad de los modelos obtenidos ante variaciones del conjunto de datos de entrenamiento, dificultad para modelar determinados tipos de problemas, etc.

Es importante comentar que los valores por defecto de los hiperparámetros que controlan el tamaño del árbol de decisión producen árboles de decisión profundos (*fully developed decision trees*) que suelen estar sobreajustados al conjunto de datos de entrenamiento. Por ello, es recomendable configurar estos hiperparámetros de acuerdo con el problema a ser resuelto.

Los árboles de decisión se encuentran implementados en la clase [`tree.DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Vamos a configurar este tipo de árbol:

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

### Un pequeño paréntesis

En general, los modelos tienen diferente capacidad para ajustarse a los datos. Intuitivamente, podemos ver que modelos más complejos permitirán un mayor ajuste a los datos, mientras que modelos sencillos tendrán más dificultades. Sin embargo, los modelos más complejos no siempre son los más adecuados debido al sobreajuste. Si el modelo está sobreajustado se dice que no generaliza ante casos nuevos.

Analizando detenidamente el error que comete un modelo se suele hacer la distinción entre dos tipos:

* **Error debido al sesgo**: Se debe a las suposiciones incorrectas del modelo. Por ejemplo, una regresión lineal supone que la variable objetivo se relaciona con las variables predictoras de acuerdo con un patrón lineal. Si por el contrario los datos siguieran otra distribución, la suposición del modelo sería incorrecta, lo que llevaría a un error predictivo. Este error suele reducirse usando modelos más complejos o una mayor dimensionalidad de los datos.

* **Error debido a la varianza**: Se produce cuando el modelo aprende patrones espúreos debido al ruido de los datos. Por ejemplo, el algoritmo del vecino más cercano. Este modelo tiene una gran capacidad de sobreajuste ya que utiliza el ejemplo más cercano del conjunto de datos de entrenamiento para realizar la predicción. Por ello, la existencia de ruido en el conjunto de datos de entrenamiento le afecta en gran medida, ya que los ejemplos nuevos que se encuentren cerca de ejemplos ruidosos se predecirán de forma incorrecta. Este error suele reducirse aumentando el número de ejemplos del conjunto de datos de entrenamiento o usando modelos más simples.

De la explicación anterior se puede entrever que existen modelos con un alto sesgo (y que gran parte del error que cometan se deberá a este) y modelos con alta varianza (que se ven afectados por patrones espúreos). Es interesante saber las características de nuestro modelo para saber si tiene más varianza o sesgo y así poder proceder adecuadamente en la reducción del error.

---

**Palabras clave**:

* Decimos que un modelo está sobreajustado cuando este no solo aprende relaciones reales sino que también se ajusta a patrones ruidosos no relacionados con la distribución subyacente de los datos.

---

## 3.3. *Adaptive Boosting*

Aunque la estrategia común detrás de un ensemble sea la de generar un consenso entre varios modelos, la forma en la que se generan está dirigida por motivaciones muy diferentes. Cuando utilizamos una técnica de ensemble es básico conocer el fundamento interno para seleccionar adecuadamente los hiperparámetros del algoritmo de aprendizaje.

En el caso de este tipo de algoritmos, la estrategia es, dado un problema complejo, reducir el sesgo de los modelos. Por ello, lo que se hace es guiar la función de aprendizaje para que incida en aquellos ejemplos que sean más dificiles de generalizar. La técnica es pesar las instancias para sobreajustar el clasificador en aquellos casos conflictivos. De manera más formal se trata de establecer un problema de optimización de funciones de coste.

Por ello, los clasificadores ideales que se pueden utilizar en este tipo de técnicas son clasificadores con un poder de generalización y parámetros no muy complejos ni sobreajustados (*weak learners*). Esto otorga al algoritmo espacio para optimizar dichos parámetros y teóricamente transformar el clasificador base en un *strong learner*. Por ejemplo, árboles de decisión poco profundos (*shallow decision trees*).

Este clasificador está disponible en la clase [`ensemble.AdaBoostClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html). Vamos a configurarlo para que utilice los hiperparámetros por defecto:



In [24]:
from sklearn.ensemble import AdaBoostClassifier

In [25]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

## 3.4. *Bootstrap Aggregating*

En este tipo de ensembles, la estrategia es utilizar una función de aprendizaje y obtener modelos bien diversos entre sí para reducir el error obtenido mediante varianza. Formalmente, hablamos de clasificadores que individualmente tengan poder de generalización y un gran poder predictivo, pero que al mismo tiempo estén lo menos correlados entre sí. Por ello, se utilizan diversas técnicas de muestreo y aleatorización de los modelos. De esta manera, el mejor tipo de clasificadores que se pueden utilizar son *strong learners*, esto es, clasificadores con un gran poder predictivo en sus parámetros y modelos que presenten mucha varianza para reducirla mediante agregación (p.e., árboles de decisión profundos).

Este clasificador se encuentra implementado en la clase [`ensemble.BaggingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html). Vamos a configurarlo con los hiperparámetros por defecto:

In [26]:
from sklearn.ensemble import BaggingClassifier

In [27]:
bagging_model = BaggingClassifier(random_state=random_state)

## 3.5. *Random Forests*

En este caso, también se busca reducir el error obtenido mediante varianza, pero adicionalmente integra otras técnicas de aleatorización en el aprendizaje de los árboles de decisión para ampliar la generalización del ensemble. Concretamente, utiliza una muestra aleatoria de los atributos a la hora de seleccionar cada punto óptimo de corte para reducir la varianza de cada uno de los estimadores del ensemble, a costa de incrementar ligeramente el sesgo. Por esa misma razón, los hiperparámetros de este algoritmo incluyen tanto los correspondientes al ensemble como al árbol de decisión.

Este ensemble se implementa en la clase [`ensemble.RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html):

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
random_forest_model = RandomForestClassifier(random_state=random_state)

## 3.6. *Gradient Tree Boosting*

Este ensemble es una generalización de los algoritmos de *boosting* con la capacidad de optimizar cualquier tipo de función pérdida. Esto permite extender estos algoritmos más allá de problemas de clasificación binaria tal y como pueden ser problemas de regresión, etc.

Este clasificador está implementado en la clase [`ensemble.GradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html):

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

## 3.7. *Histogram Gradient Boosting*

Este algoritmo es una optimización del previo que discretiza el conjunto de datos de entrada para reducir el número de puntos de corte a considerar en la construcción de los árboles de decisión (aprovechándose de estructuras de datos basadas en histogramas). De esta manera, no tiene que considerar cada valor distinto de las variables predictoras continuas como punto de corte al construir los árboles de decisión, lo que le permite reducir en varios órdenes de magnitud el tiempo de entrenamiento e inferencia. Otra de las ventajas que presenta es que es capaz de tratar valores perdidos de manera implícita sin necesidad de realizar una imputación previa.

Este ensemble se implementa en la clase [`ensemble.HistGradientBoostingClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html):

In [32]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [33]:
hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

# 4. Evaluación de modelos

Hemos visto que a la hora de entrenar un modelo podemos parametrizar el algoritmo de aprendizaje, esto es, configurar sus hiperparámetros. Llegados a este punto, nos deben surgir una serie de dudas:

* ¿Cuál es el mejor clasificador para mi problema?
* ¿Cuál es la mejor configuración de hiperparámetros para mi problema?

Para responder a estas preguntas, lo más lógico sería realizar una serie de experimentos para evaluar distintos modelos y quedarnos con el mejor. No obstante, hemos de realizar dichas pruebas con una metodología que garantice que los resultados no están sesgados.

Una de lás técnicas más utilizadas es la validación cruzada. En esta, se separa el conjunto de datos en $ k $ particiones y se repite $ k $ veces el proceso de aprendizaje y validación, pero utilizando cada vez una combinación única de $ k - 1 $ muestras para entrenar y la restante para validar. El resultado es la media de las métricas obtenidas en cada una de las particiones. Al repetir y agregar los resultados del aprendizaje sobre varios conjuntos de datos diferentes, nos aseguramos una buena estimación del sesgo y la varianza.

Vamos a configurar una $ 5 \times 10 $ validación cruzada estratificada, para lo cuál recurrimos a la clase [`sklearn.model_selection.RepeatedStratifiedKFold`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html):

---

**Notas**:

* En el siguiente [enlace](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators) se proporciona el listado completo de tipos de validación cruzada que se pueden aplicar.

---

In [34]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [35]:
n_splits = 10

In [36]:
n_repeats = 5

In [37]:
cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)

# 5. Selección de modelos

Ahora que somos capaces de evaluar correctamente los clasificadores es importante decidir una estrategia que nos permita encontrar una configuración óptima de los hiperparámetros. Para ello, combinaremos un algoritmo de búsqueda con evaluación mediante validación cruzada, y así explorar el espacio de las distintas configuraciones de hiperparámetros que proveen los algoritmos de aprendizaje.

Cuando se tiene cierta experiencia usando los modelos es posible determinar los dominios aproximados en los que se encuentran los hiperparámetros relevantes que puede tomar un algoritmo de aprendizaje. No obstante, es un proceso de ensayo, error e intuición que debemos adquirir (así que es razonable explorar configuraciones y estudiar el resultado).

El algoritmo más básico de selección de modelos por fuerza bruta se llama búsqueda en malla y está implementado en [`model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Este recibe un conjunto de hiperparámetros (diccionario con el nombre del hiperparámetro como clave y la lista de configuraciones como valores) y realiza una búsqueda exhaustiva evaluando mediante validación cruzada todas las posibles combinaciones existentes.

En el algoritmo de los vecinos más cercanos parece lógico optimizar el número de vecinos (`n_neighbors`) y la función de pesado (`weights`):

In [38]:
from utils import optimize_params

In [39]:
weights = ["uniform", "distance"]

In [40]:
n_neighbors = [1, 2, 3, 4, 5, 6, 7, 8]

In [41]:
k_neighbors_classifier = optimize_params(k_neighbors_model, X_train, y_train, cv, weights=weights, n_neighbors=n_neighbors)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
15,0.002130,0.000217,0.002274,0.000283,8,distance,"{'n_neighbors': 8, 'weights': 'distance'}",0.977273,0.040503,1,1.000000,0.000000
10,0.002277,0.000382,0.002663,0.000528,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.975455,0.045281,2,0.978193,0.007197
9,0.002184,0.000345,0.002271,0.000291,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.975455,0.041471,3,1.000000,0.000000
8,0.002233,0.000502,0.002634,0.000535,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.973636,0.046078,4,0.978831,0.006011
14,0.002294,0.000420,0.002649,0.000479,8,uniform,"{'n_neighbors': 8, 'weights': 'uniform'}",0.973636,0.046078,4,0.977344,0.008519
4,0.002296,0.000569,0.002652,0.000603,3,uniform,"{'n_neighbors': 3, 'weights': 'uniform'}",0.973636,0.042339,6,0.972493,0.008186
5,0.002363,0.000533,0.002462,0.000502,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.973636,0.042339,6,1.000000,0.000000
11,0.002448,0.000606,0.002656,0.000894,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.973455,0.047092,8,1.000000,0.000000
13,0.002476,0.000599,0.002588,0.000611,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.973455,0.046349,8,1.000000,0.000000
6,0.002828,0.001249,0.003192,0.001032,4,uniform,"{'n_neighbors': 4, 'weights': 'uniform'}",0.971818,0.043112,10,0.972916,0.007365


La mejor configuración de hiperparámetros es un número de vecinos más cercanos moderado y pesar de acuerdo con la inversa de la distancia.

Continuando con los árboles de decisión, podemos optimizar el criterion de partición (`criterion`), la altura máxima (`max_depth`) y el parámetro de complejidad de la poda (`ccp_alpha`):

In [42]:
criterion = ["gini", "entropy"]

In [43]:
max_depth = [1, 2, 3, 4, 5, 6, None]

In [44]:
ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]

In [45]:
decision_tree_classifier = optimize_params(decision_tree_model, X_train, y_train, cv, criterion=criterion, max_depth=max_depth, ccp_alpha=ccp_alpha)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ccp_alpha,param_criterion,param_max_depth,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
9,0.002387,0.000391,0.001635,0.000282,0.0,entropy,3,"{'ccp_alpha': 0.0, 'criterion': 'entropy', 'ma...",0.932545,0.074547,1,0.973348,0.007664
2,0.002242,0.000069,0.001561,0.000090,0.0,gini,3,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.930727,0.076188,2,0.973774,0.007946
5,0.002229,0.000059,0.001555,0.000060,0.0,gini,6,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.929091,0.079357,3,1.000000,0.000000
6,0.002500,0.000525,0.001732,0.000343,0.0,gini,None,"{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_d...",0.929091,0.079357,3,1.000000,0.000000
10,0.002724,0.000725,0.001821,0.000448,0.0,entropy,4,"{'ccp_alpha': 0.0, 'criterion': 'entropy', 'ma...",0.927273,0.078309,5,0.989637,0.007486
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.002442,0.000360,0.001675,0.000273,0.4,gini,4,"{'ccp_alpha': 0.4, 'criterion': 'gini', 'max_d...",0.286364,0.013636,64,0.338634,0.001792
60,0.002306,0.000095,0.001551,0.000055,0.4,gini,5,"{'ccp_alpha': 0.4, 'criterion': 'gini', 'max_d...",0.286364,0.013636,64,0.338634,0.001792
61,0.002405,0.000356,0.001596,0.000213,0.4,gini,6,"{'ccp_alpha': 0.4, 'criterion': 'gini', 'max_d...",0.286364,0.013636,64,0.338634,0.001792
62,0.002428,0.000375,0.001708,0.000364,0.4,gini,None,"{'ccp_alpha': 0.4, 'criterion': 'gini', 'max_d...",0.286364,0.013636,64,0.338634,0.001792


Lo que podemos observar es que los mejores hiperparámetros son podar con una altura máxima del árbol decisión de 3 y usar la ganancia de información como criterio de partición.

En *Adaptive Boosting* vamos a optimizar la tasa de aprendizaje (`learning_rate`) y el número de estimadores del ensemble (`n_estimators`). Además, podemos optimizar el criterio de partición y la altura máxima del árbol de decisión que vamos a usar como estimador base:

---

**Nota**: Se pueden optimizar los hiperparámetros de estimadores compuestos tal y como se detalla en este [enlace](https://scikit-learn.org/stable/modules/grid_search.html#composite-estimators-and-parameter-spaces).

---

In [46]:
base_estimator = DecisionTreeClassifier(random_state=random_state)

In [47]:
base_estimator = [base_estimator]

In [48]:
n_estimators = [20, 50, 100]

In [49]:
learning_rate = [0.95, 1.0]

In [50]:
max_depth = [1, 2, 3]

In [51]:
adaboost_classifier = optimize_params(adaboost_model, X_train, y_train, cv, base_estimator=base_estimator, n_estimators=n_estimators, learning_rate=learning_rate, base_estimator__criterion=criterion, base_estimator__max_depth=max_depth)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base_estimator,param_base_estimator__criterion,param_base_estimator__max_depth,param_learning_rate,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
3,0.029373,0.004275,0.004693,0.001157,"DecisionTreeClassifier(max_depth=1, random_sta...",gini,1,1.0,20,{'base_estimator': DecisionTreeClassifier(max_...,0.953636,0.064340,1,0.972058,0.024028
21,0.046521,0.007869,0.007020,0.001255,"DecisionTreeClassifier(max_depth=1, random_sta...",entropy,1,1.0,20,{'base_estimator': DecisionTreeClassifier(max_...,0.947818,0.064278,2,0.971637,0.024821
33,0.024756,0.009760,0.003794,0.001033,"DecisionTreeClassifier(max_depth=1, random_sta...",entropy,3,1.0,20,{'base_estimator': DecisionTreeClassifier(max_...,0.945818,0.066704,3,1.000000,0.000000
28,0.074303,0.010204,0.008239,0.001477,"DecisionTreeClassifier(max_depth=1, random_sta...",entropy,2,1.0,50,{'base_estimator': DecisionTreeClassifier(max_...,0.945818,0.066704,3,1.000000,0.000000
25,0.073487,0.009546,0.008425,0.001870,"DecisionTreeClassifier(max_depth=1, random_sta...",entropy,2,0.95,50,{'base_estimator': DecisionTreeClassifier(max_...,0.945818,0.066704,3,1.000000,0.000000
17,0.095229,0.059927,0.010213,0.005585,"DecisionTreeClassifier(max_depth=1, random_sta...",gini,3,1.0,100,{'base_estimator': DecisionTreeClassifier(max_...,0.944000,0.068884,6,1.000000,0.000000
30,0.028336,0.010518,0.004379,0.001433,"DecisionTreeClassifier(max_depth=1, random_sta...",entropy,3,0.95,20,{'base_estimator': DecisionTreeClassifier(max_...,0.944000,0.066441,6,1.000000,0.000000
27,0.031365,0.005548,0.004793,0.001147,"DecisionTreeClassifier(max_depth=1, random_sta...",entropy,2,1.0,20,{'base_estimator': DecisionTreeClassifier(max_...,0.944000,0.068884,6,1.000000,0.000000
14,0.092390,0.063939,0.010214,0.006550,"DecisionTreeClassifier(max_depth=1, random_sta...",gini,3,0.95,100,{'base_estimator': DecisionTreeClassifier(max_...,0.944000,0.066441,9,1.000000,0.000000
32,0.098855,0.061043,0.010666,0.006160,"DecisionTreeClassifier(max_depth=1, random_sta...",entropy,3,0.95,100,{'base_estimator': DecisionTreeClassifier(max_...,0.943818,0.066549,10,1.000000,0.000000


La mejor configuración de hiperparámetros es usar un tamaño máximo del árbol de decisión de 1 y como criterio el índice *Gini*, una tasa de aprendizaje de 1.0 y 20 estimadores en el ensemble.

En *Bootstrap Aggregating* únicamente vamos a optimizar el número de estimadores del ensemble y el criterio de partición de los árboles de decisión que vamos a usar como estimador base:

In [52]:
base_estimator = DecisionTreeClassifier(random_state=random_state)

In [53]:
base_estimator = [base_estimator]

In [54]:
bagging_classifier = optimize_params(bagging_model, X_train, y_train, cv, base_estimator=base_estimator, n_estimators=n_estimators, base_estimator__criterion=criterion)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base_estimator,param_base_estimator__criterion,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
1,0.074489,0.011783,0.006720,0.001798,DecisionTreeClassifier(random_state=27912),gini,50,{'base_estimator': DecisionTreeClassifier(rand...,0.951455,0.059213,1,0.999787,0.001489
2,0.144707,0.016404,0.011065,0.002610,DecisionTreeClassifier(random_state=27912),gini,100,{'base_estimator': DecisionTreeClassifier(rand...,0.949455,0.056368,2,1.000000,0.000000
5,0.143497,0.013427,0.010770,0.003100,DecisionTreeClassifier(random_state=27912),entropy,100,{'base_estimator': DecisionTreeClassifier(rand...,0.949455,0.059228,2,1.000000,0.000000
4,0.072278,0.007713,0.006197,0.001047,DecisionTreeClassifier(random_state=27912),entropy,50,{'base_estimator': DecisionTreeClassifier(rand...,0.947636,0.059044,4,0.999787,0.001489
0,0.031117,0.005182,0.003757,0.000678,DecisionTreeClassifier(random_state=27912),gini,20,{'base_estimator': DecisionTreeClassifier(rand...,0.946000,0.066591,5,0.997675,0.005706
3,0.030230,0.003489,0.003746,0.000700,DecisionTreeClassifier(random_state=27912),entropy,20,{'base_estimator': DecisionTreeClassifier(rand...,0.944000,0.068884,6,0.997254,0.005903


Sin mucho misterio, la mejor configuración de hiperparámetros es usar el índice *Gini* como criterio de partición en los árboles de decisión y un total de 50 estimadores en el ensemble.

En *Random Forests* vamos a optimizar los mismos hiperparámetros que para *Bootstrap Aggregating*, pero añadiendo el número máximo de características a considerar en cada nodo de los árboles de decisión:

In [55]:
max_features = ["sqrt", "log2"]

In [56]:
random_forest_classifier = optimize_params(random_forest_model, X_train, y_train, cv, n_estimators=n_estimators, criterion=criterion, max_features=max_features)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
1,0.080124,0.023313,0.008311,0.002483,gini,sqrt,50,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.953091,0.056579,1,1.000000,0.000000
4,0.059542,0.005646,0.006519,0.001223,gini,log2,50,"{'criterion': 'gini', 'max_features': 'log2', ...",0.953091,0.056579,1,1.000000,0.000000
2,0.138925,0.039613,0.013411,0.004573,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.951273,0.056503,3,1.000000,0.000000
5,0.121629,0.015707,0.010635,0.001772,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.951273,0.056503,3,1.000000,0.000000
8,0.125775,0.017446,0.011163,0.002164,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.951273,0.056503,3,1.000000,0.000000
11,0.120706,0.012338,0.010861,0.002057,entropy,log2,100,"{'criterion': 'entropy', 'max_features': 'log2...",0.951273,0.056503,3,1.000000,0.000000
0,0.026431,0.004589,0.004061,0.000727,gini,sqrt,20,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.949455,0.061956,7,0.998730,0.003439
3,0.026281,0.004321,0.003920,0.000875,gini,log2,20,"{'criterion': 'gini', 'max_features': 'log2', ...",0.949455,0.061956,7,0.998730,0.003439
7,0.060912,0.007492,0.006706,0.001132,entropy,sqrt,50,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.949273,0.059366,9,1.000000,0.000000
10,0.063784,0.010219,0.007352,0.001862,entropy,log2,50,"{'criterion': 'entropy', 'max_features': 'log2...",0.949273,0.059366,9,1.000000,0.000000


Los mejores hiperparámetros son usar el índice *Gini* como criterio de partición, únicamente considerar la raíz del número de características en cada nodo y un total de 50 estimadores en el ensemble.

Continuamos con la optimización de hiperparámetros para *Gradient Tree Boosting*. Vamos a probar el parámetro de regularización (`learning_rate`), el número de estimadores del ensemble y el criterio de partición y la altura máxima de los árboles de decisión base:

In [57]:
learning_rate = [0.01, 0.05, 0.1]

In [58]:
criterion = ["friedman_mse", "squared_error"]

In [59]:
gradient_boosting_classifier = optimize_params(gradient_boosting_model, X_train, y_train, cv, learning_rate=learning_rate, n_estimators=n_estimators, criterion=criterion, max_depth=max_depth)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_learning_rate,param_max_depth,param_n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
34,0.096070,0.010888,0.002550,0.000534,squared_error,0.01,3,50,"{'criterion': 'squared_error', 'learning_rate'...",0.946000,0.071383,1,0.993234,0.005075
7,0.147358,0.016821,0.003671,0.000848,friedman_mse,0.01,3,50,"{'criterion': 'friedman_mse', 'learning_rate':...",0.946000,0.071383,1,0.993234,0.005075
41,0.211096,0.049863,0.003075,0.000665,squared_error,0.05,2,100,"{'criterion': 'squared_error', 'learning_rate'...",0.943818,0.067068,3,1.000000,0.000000
22,0.134073,0.014913,0.003847,0.001144,friedman_mse,0.1,2,50,"{'criterion': 'friedman_mse', 'learning_rate':...",0.943818,0.067068,3,1.000000,0.000000
49,0.086746,0.010089,0.002508,0.000381,squared_error,0.1,2,50,"{'criterion': 'squared_error', 'learning_rate'...",0.943818,0.067068,3,1.000000,0.000000
14,0.207608,0.042866,0.003320,0.001909,friedman_mse,0.05,2,100,"{'criterion': 'friedman_mse', 'learning_rate':...",0.943818,0.067068,3,1.000000,0.000000
33,0.039895,0.004654,0.002274,0.000463,squared_error,0.01,3,20,"{'criterion': 'squared_error', 'learning_rate'...",0.942182,0.066127,7,0.986253,0.007693
6,0.059750,0.007770,0.003324,0.000819,friedman_mse,0.01,3,20,"{'criterion': 'friedman_mse', 'learning_rate':...",0.942182,0.066127,7,0.986676,0.007857
8,0.289091,0.025164,0.004007,0.001220,friedman_mse,0.01,3,100,"{'criterion': 'friedman_mse', 'learning_rate':...",0.942000,0.069184,9,0.994714,0.005286
21,0.054059,0.008954,0.003432,0.000696,friedman_mse,0.1,2,20,"{'criterion': 'friedman_mse', 'learning_rate':...",0.942000,0.066752,9,0.993651,0.005994


En este caso, son varias las mejores configuraciones de hiperparámetros, pero la seleccionada es usar como criterio de partición el error cuadrático medio con la mejora de *Friedman*, un tamaño de árbol de 3, una regularización de 0.01 y 50 estimadores en el ensemble.

Para finalizar, vamos a optimizar el parámetro de regularización, el número de iteraciones del proceso (`max_iter`) y el número máximo de nodos hoja (`max_leaf_nodes`) de los árboles de decisión en el algoritmo *Histogram Gradient Boosting*:

In [60]:
learning_rate = [0.01, 0.02, 0.03, 0.04, 0.05]

In [61]:
max_iter = n_estimators

In [62]:
max_leaf_nodes = [15, 31, 65, 127]

In [63]:
hist_gradient_boosting_classifier = optimize_params(hist_gradient_boosting_model, X_train, y_train, cv, learning_rate=learning_rate, max_iter=max_iter, max_leaf_nodes=max_leaf_nodes)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_iter,param_max_leaf_nodes,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
55,0.556036,1.936325,0.086826,0.340646,0.05,50,127,"{'learning_rate': 0.05, 'max_iter': 50, 'max_l...",0.945636,0.066816,1,0.986670,0.008132
54,0.113505,0.015639,0.005915,0.003818,0.05,50,65,"{'learning_rate': 0.05, 'max_iter': 50, 'max_l...",0.945636,0.066816,1,0.986670,0.008132
53,0.370242,1.121637,0.007508,0.005041,0.05,50,31,"{'learning_rate': 0.05, 'max_iter': 50, 'max_l...",0.945636,0.066816,1,0.986670,0.008132
52,0.114879,0.015063,0.005599,0.002274,0.05,50,15,"{'learning_rate': 0.05, 'max_iter': 50, 'max_l...",0.945636,0.066816,1,0.986670,0.008132
23,0.226128,0.023522,0.007980,0.000780,0.02,100,127,"{'learning_rate': 0.02, 'max_iter': 100, 'max_...",0.943818,0.071343,5,0.977373,0.011161
22,0.223752,0.023828,0.011429,0.008124,0.02,100,65,"{'learning_rate': 0.02, 'max_iter': 100, 'max_...",0.943818,0.071343,5,0.977373,0.011161
21,0.223080,0.019716,0.014867,0.028513,0.02,100,31,"{'learning_rate': 0.02, 'max_iter': 100, 'max_...",0.943818,0.071343,5,0.977373,0.011161
20,0.222495,0.020733,0.010833,0.006970,0.02,100,15,"{'learning_rate': 0.02, 'max_iter': 100, 'max_...",0.943818,0.071343,5,0.977373,0.011161
32,0.221329,0.015923,0.010368,0.007073,0.03,100,15,"{'learning_rate': 0.03, 'max_iter': 100, 'max_...",0.943818,0.066549,5,0.989848,0.004703
33,0.289166,0.440582,0.011372,0.008911,0.03,100,31,"{'learning_rate': 0.03, 'max_iter': 100, 'max_...",0.943818,0.066549,5,0.989848,0.004703


Así, se obtiene que la mejor configuración de hiperparámetros es una regularización de 0.05, 50 iteraciones y un máximo de 15 nodos hoja.

# 6. Construcción y validación del modelo final

En un entorno de producción se debe construir un clasificador que será el que se despliegue y use para clasificar nuevas instancias. Es importante saber que, después de un proceso de validación cruzada para la optimización de los hiperparámetros, se debe obtener un modelo final y una medida generalizada de su rendimiento.

Para ello, nos surgen dos dudas:

* La validación cruzada crea `k` modelos, ¿con cuál nos quedamos?
* La medida de rendimiento obtenida es ¿la media de la validación cruzada? o ¿para una partición concreta?

La solución pasa por utilizar el conjunto de datos de prueba que se ha quedado fuera de la validación cruzada. De este modo, realizaremos la validación cruzada a partir del conjunto de datos de entrenamiento para seleccionar nuestro modelo. Una vez se ha obtenido la mejor configuración de hiperparámetros (de acuerdo con la métrica de rendimiento utilizada durante este proceso), utilizaremos el conjunto de datos de entrenamiento para entrenar un nuevo clasificador final a partir de dicha configuración. Para validar este clasificador y obtener una métrica no sesgada de su rendimiento, utilizaremos el conjunto de datos de prueba que separamos al principio y que constituye una muestra de datos nunca visualizada por el  algoritmo de aprendizaje.

Esta última fase de entrenamiento es realizada automáticamente por `model_selection.GridSearchCV`, por lo que podemos utilizarlo directamente para obtener una métrica de rendimiento final para cada uno de los clasificadores:

In [64]:
from utils import evaluate_estimators

In [65]:
estimators = [k_neighbors_classifier, decision_tree_classifier, adaboost_classifier, bagging_classifier, random_forest_classifier, gradient_boosting_classifier, hist_gradient_boosting_classifier]

In [66]:
evaluate_estimators(estimators, X_test, y_test, "accuracy")

,accuracy
KNeighborsClassifier,0.955556
DecisionTreeClassifier,0.977778
AdaBoostClassifier,0.977778
BaggingClassifier,0.977778
RandomForestClassifier,0.977778
GradientBoostingClassifier,0.977778
HistGradientBoostingClassifier,0.977778


De acuerdo con estos resultados, podemos concluir que los árboles de decisión y los ensembles son los clasificadores que ofrecen mejor rendimiento (en términos de tasa de acierto) en el conjunto de datos `iris`.

## Un pequeño paréntesis: Más allá de las métricas de rendimiento

Si bien nos hemos focalizado en evaluar el rendimiento de los clasificadores usando la tasa de acierto, también es importante tener en cuenta otros factores como es el tiempo de aprendizaje e inferencia. De esta manera, si bien hemos concluido que los árboles de decisión y los ensembles son los clasificadores que ofrecen un mejor rendimiento (en términos de tasa de acierto), los árboles de decisión tienen un tiempo de aprendizaje e inferencia mucho menor que los ensembles. Es por ello por lo que se deben analizar los recursos disponibles y en base a esto decidir si merece la pena incrementar el coste computacional a cambio de conseguir un mejor rendimiento (en términos de la métrica elegida para el problema en cuestión).

# Trabajo autónomo

#### 1. Lectura y estudio

En esta práctica se debe estudiar cuidadosamente la documentación de esta libreta y leer las páginas de ejemplo y teoría de los distintos estimadores y métodos de validación del manual de `scikit-learn`. Encontraréis que es una lectura ligera y muy instructiva:

* [Vecinos más cercanos](https://scikit-learn.org/stable/modules/neighbors.html)
* [Árboles de decisión](https://scikit-learn.org/stable/modules/tree.html)
* [*Adaptive Boosting*](https://scikit-learn.org/stable/modules/ensemble.html#adaboost)
* [*Bootstrap Aggregating*](https://scikit-learn.org/stable/modules/ensemble.html#bagging)
* [*Random Forests*](https://scikit-learn.org/stable/modules/ensemble.html#random-forests)
* [*Gradient Tree Boosting*](https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting)
* [*Histogram Gradient Boosting*](https://scikit-learn.org/stable/modules/ensemble.html#histogram-based-gradient-boosting)
* [Validación cruzada](http://scikit-learn.org/stable/modules/cross_validation.html)
* [Búsqueda en malla](http://scikit-learn.org/stable/modules/grid_search.html)

#### 2. Estudio de una libreta

Navegar por los distintos conjuntos de datos públicos que hay en *Kaggle* y ver las libretas más populares. Seleccionar uno y estudiarlo. Tendréis que presentarlo en la defensa de la práctica (y se valorará positivamente reproducirlo en vuestro propio entorno). Las libretas se encuentran en este [enlace](https://www.kaggle.com/kernels).

#### 3. Entregable

Deberéis entregar un informe reproduciendo el estudio sobre el conjunto de datos `titanic`. Incluir un enlace a la libreta de *Kaggle* seleccionada (o el estudio si se ha decidido reproducirlo). En la evaluación se valorará, a parte de esto, el estudio que hagáis tanto de los métodos propuestos así como de la libreta de *Kaggle* de vuestra elección.

**Recordar que asociada a esta práctica habrá una entrevista para completar la evaluación de la práctica.**

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=bda2ffb5-8e8f-47a2-bfda-c8bad15068dc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>